<a href="https://colab.research.google.com/github/sasidharperla/Building-a-Simple-Chatbot-in-Python-using-NLTK/blob/master/automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import json
import random
from datetime import datetime, timedelta

In [2]:
def generate_synthetic_data(num_samples=1000):
    """Generate synthetic home automation training data"""
    data = []

    for _ in range(num_samples):
        # Time features
        hour = random.randint(0, 23)
        weekday = random.randint(0, 6)  # 0=Monday
        month = random.randint(1, 12)

        # Environmental conditions
        temp = random.uniform(15, 35)
        humidity = random.uniform(30, 80)
        light_level = random.uniform(0, 100)

        # Occupancy (simulated motion sensor)
        occupied = random.choice([0, 1])

        # Generate labels (what actions should happen)
        lights_on = 0
        ac_on = 0
        security_mode = 0

        # Rules to create patterns (these are what the ML will learn)
        if occupied and (hour > 18 or hour < 6) and light_level < 30:
            lights_on = 1
        if temp > 25:
            ac_on = 1
        if not occupied and (hour > 22 or hour < 6):
            security_mode = 1

        data.append([
            hour, weekday, month, temp, humidity, light_level, occupied,
            lights_on, ac_on, security_mode
        ])

    columns = [
        'hour', 'weekday', 'month', 'temperature', 'humidity', 'light_level', 'occupied',
        'lights_on', 'ac_on', 'security_mode'
    ]

    return pd.DataFrame(data, columns=columns)

In [3]:
def train_models(df):
    """Train separate models for each automation task"""
    X = df[['hour', 'weekday', 'month', 'temperature', 'humidity', 'light_level', 'occupied']]

    # Model for lights
    y_lights = df['lights_on']
    X_train, X_test, y_train, y_test = train_test_split(X, y_lights, test_size=0.2)
    lights_model = RandomForestClassifier()
    lights_model.fit(X_train, y_train)
    lights_acc = accuracy_score(y_test, lights_model.predict(X_test))

    # Model for AC
    y_ac = df['ac_on']
    X_train, X_test, y_train, y_test = train_test_split(X, y_ac, test_size=0.2)
    ac_model = RandomForestClassifier()
    ac_model.fit(X_train, y_train)
    ac_acc = accuracy_score(y_test, ac_model.predict(X_test))

    # Model for security
    y_security = df['security_mode']
    X_train, X_test, y_train, y_test = train_test_split(X, y_security, test_size=0.2)
    security_model = RandomForestClassifier()
    security_model.fit(X_train, y_train)
    security_acc = accuracy_score(y_test, security_model.predict(X_test))

    print(f"Model accuracies - Lights: {lights_acc:.2f}, AC: {ac_acc:.2f}, Security: {security_acc:.2f}")

    return {
        'lights_model': lights_model,
        'ac_model': ac_model,
        'security_model': security_model
    }

In [5]:
class VirtualHome:
    def __init__(self, models):
        self.models = models
        self.state = {
            'lights': False,
            'ac': False,
            'security_mode': False,
            'temperature': 22,
            'humidity': 45,
            'light_level': 50,
            'occupied': False
        }
        self.log = []

    def update_sensors(self, temp=None, humidity=None, light=None, occupied=None):
        """Simulate sensor updates"""
        if temp is not None:
            self.state['temperature'] = temp
        if humidity is not None:
            self.state['humidity'] = humidity
        if light is not None:
            self.state['light_level'] = light
        if occupied is not None:
            self.state['occupied'] = occupied

    def get_current_features(self):
        """Prepare feature vector for prediction"""
        now = datetime.now()
        return [
            now.hour,
            now.weekday(),
            now.month,
            self.state['temperature'],
            self.state['humidity'],
            self.state['light_level'],
            self.state['occupied']
        ]

    def run_automation(self):
        """Make predictions and update home state"""
        features = [self.get_current_features()]

        # Predict actions
        lights_pred = self.models['lights_model'].predict(features)[0]
        ac_pred = self.models['ac_model'].predict(features)[0]
        security_pred = self.models['security_model'].predict(features)[0]

        # Update state
        self.state['lights'] = bool(lights_pred)
        self.state['ac'] = bool(ac_pred)
        self.state['security_mode'] = bool(security_pred)

        # Log action
        action = {
            'timestamp': datetime.now().isoformat(),
            'actions': {
                'lights': self.state['lights'],
                'ac': self.state['ac'],
                'security_mode': self.state['security_mode']
            },
            'sensor_state': self.state.copy()
        }
        self.log.append(action)

        return action

In [6]:
if __name__ == "__main__":
    # Step 1: Generate and train on synthetic data
    print("Generating training data...")
    df = generate_synthetic_data(5000)

    print("Training models...")
    models = train_models(df)

    # Step 2: Create virtual home with trained models
    print("\nCreating virtual smart home...")
    smart_home = VirtualHome(models)

    # Simulate a day (you would normally get this from real sensors)
    print("\nSimulating a day in the virtual home...")
    times_of_day = [
        (6, 20, 45, 60, True),   # Morning
        (12, 28, 40, 80, True),  # Afternoon (hot)
        (19, 24, 50, 30, True),  # Evening
        (23, 22, 55, 10, False)  # Night (unoccupied)
    ]

    for hour, temp, humidity, light, occupied in times_of_day:
        print(f"\n--- {hour}:00 ---")
        smart_home.update_sensors(temp=temp, humidity=humidity,
                                 light=light, occupied=occupied)
        action = smart_home.run_automation()

        print(f"Sensor State: Temp={temp}C, Light={light}%, Occupied={occupied}")
        print(f"Actions: Lights={'ON' if action['actions']['lights'] else 'OFF'}, "
              f"AC={'ON' if action['actions']['ac'] else 'OFF'}, "
              f"Security={'ON' if action['actions']['security_mode'] else 'OFF'}")

Generating training data...
Training models...
Model accuracies - Lights: 1.00, AC: 1.00, Security: 1.00

Creating virtual smart home...

Simulating a day in the virtual home...

--- 6:00 ---
Sensor State: Temp=20C, Light=60%, Occupied=True
Actions: Lights=OFF, AC=OFF, Security=OFF

--- 12:00 ---
Sensor State: Temp=28C, Light=80%, Occupied=True
Actions: Lights=OFF, AC=ON, Security=OFF

--- 19:00 ---
Sensor State: Temp=24C, Light=30%, Occupied=True
Actions: Lights=OFF, AC=OFF, Security=OFF

--- 23:00 ---
Sensor State: Temp=22C, Light=10%, Occupied=False
Actions: Lights=OFF, AC=OFF, Security=OFF


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local